In [28]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.layers.merge import concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, BatchNormalization, Dense, Conv2D, MaxPool2D , Flatten, Dropout, GlobalAveragePooling2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf

In [14]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [15]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [16]:
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

GPUs:  1


In [17]:
train = pd.read_csv('train_metadata.csv')
test = pd.read_csv('test_metadata.csv')
#submission = pd.read_csv('sample_submission.csv')

train_dir = 'train/'
test_dir = 'test/'

In [18]:
label = []
train_data = []
for i in range(train.shape[0]):
    train_data.append(train_dir + train['image_name'].iloc[i] + '.jpg')
    label.append(train['diagnosis'].iloc[i])
    
test_data = []
for i in range(test.shape[0]):
    test_data.append(test_dir + test['image'].iloc[i] + '.jpg')

In [19]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
label_df = pd.DataFrame(label, columns = ['diagnosis'])
one_hot_encoder.fit(label_df)

label_df_encoded = one_hot_encoder.transform(label_df)
label_df_encoded = pd.DataFrame(data=label_df_encoded, columns=one_hot_encoder.categories_)
print(label_df_encoded.columns)

MultiIndex([('atypical melanocytic proliferation',),
            (               'cafe-au-lait macule',),
            (                       'lentigo NOS',),
            (               'lichenoid keratosis',),
            (                          'melanoma',),
            (                             'nevus',),
            (              'seborrheic keratosis',),
            (                     'solar lentigo',),
            (                           'unknown',)],
           )


In [20]:
df_train = pd.DataFrame(train_data)
df_train.columns = ['images']
df_train = pd.concat([df_train, label_df_encoded], axis = 1)
print(df_train.columns)
df_test = pd.DataFrame(test_data)
df_test.columns = ['images']

Index([                               'images',
       ('atypical melanocytic proliferation',),
                      ('cafe-au-lait macule',),
                              ('lentigo NOS',),
                      ('lichenoid keratosis',),
                                 ('melanoma',),
                                    ('nevus',),
                     ('seborrheic keratosis',),
                            ('solar lentigo',),
                                  ('unknown',)],
      dtype='object')


In [21]:
X_train, X_test, y_train, y_test = train_test_split(df_train.iloc[:, 0], df_train.iloc[:,1::], test_size = 0.3, random_state = 42)

train = pd.DataFrame(X_train)
train.columns = ['images']
train = pd.concat([train, y_train], axis = 1)

test = pd.DataFrame(X_test)
test.columns = ['images']
test = pd.concat([test, y_test], axis = 1)

In [22]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=50,
                                   zoom_range = 0.3,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range = 0.2,
                                   horizontal_flip=True,
                                   fill_mode = 'nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col=train.columns[0],
    y_col=train.columns[1:],
    target_size=(224, 224),
    batch_size=8,
    shuffle=True,
    class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(
    test,
    x_col=test.columns[0],
    y_col=test.columns[1:],
    target_size=(224, 224),
    shuffle=False,
    batch_size=8,
    class_mode='raw')

Found 23188 validated image filenames.
Found 9938 validated image filenames.


In [30]:
model_d=DenseNet169(weights='imagenet',include_top=False, input_shape=(224, 224, 3)) 


x=model_d.output

x= GlobalAveragePooling2D()(x)

x= BatchNormalization()(x)

x= Dropout(0.5)(x)

x= Dense(1024,activation='relu')(x) 

x= Dense(512,activation='relu')(x) 

x= BatchNormalization()(x)

x= Dropout(0.5)(x)

class_num = 9

preds=Dense(class_num,activation='softmax')(x) #FC-layer
model=Model(inputs=model_d.input,outputs=preds)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_10 (ZeroPadding2 (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_10[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [31]:
for layer in model.layers[:-8]:

    layer.trainable=False

for layer in model.layers[-8:]:

    layer.trainable=True

In [33]:
opt = Adam(lr=0.0001)

model.compile(optimizer= opt,loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_10 (ZeroPadding2 (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_10[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [34]:
batch_size = 128
epochs = 100
nb_train_steps = train.shape[0] // batch_size
nb_val_steps = test.shape[0] // batch_size

checkpoint = ModelCheckpoint("dense-169.h5", monitor='val_accuracy', verbose=5, save_best_only=True, save_weights_only=False, mode='auto')

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

model.fit_generator(train_generator, steps_per_epoch=nb_train_steps, epochs=epochs,
                              validation_data=test_generator, validation_steps=nb_val_steps, 
                              verbose=1, callbacks=[checkpoint,early])

/home/deeplab/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-01-10 08:49:56.649289: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-10 08:49:56.666045: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 4200000000 Hz


Epoch 1/100


2022-01-10 08:50:01.394528: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-01-10 08:50:01.552078: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-01-10 08:50:01.814581: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


181/181 [==============================] - 92s 478ms/step - loss: 3.1898 - accuracy: 0.1702 - val_loss: 1.4665 - val_accuracy: 0.6802

Epoch 00001: val_accuracy improved from -inf to 0.68019, saving model to dense-169.h5
Epoch 2/100
181/181 [==============================] - 85s 466ms/step - loss: 2.5052 - accuracy: 0.3340 - val_loss: 1.2386 - val_accuracy: 0.7273

Epoch 00002: val_accuracy improved from 0.68019 to 0.72727, saving model to dense-169.h5
Epoch 3/100
181/181 [==============================] - 88s 485ms/step - loss: 1.9821 - accuracy: 0.4378 - val_loss: 1.0226 - val_accuracy: 0.7938

Epoch 00003: val_accuracy improved from 0.72727 to 0.79383, saving model to dense-169.h5
Epoch 4/100
181/181 [==============================] - 85s 471ms/step - loss: 1.8356 - accuracy: 0.5238 - val_loss: 1.0157 - val_accuracy: 0.7808

Epoch 00004: val_accuracy did not improve from 0.79383
Epoch 5/100
181/181 [==============================] - 86s 476ms/step - loss: 1.6618 - accuracy: 0.5237 -

181/181 [==============================] - 85s 472ms/step - loss: 0.4034 - accuracy: 0.8818 - val_loss: 0.3336 - val_accuracy: 0.9058

Epoch 00039: val_accuracy did not improve from 0.91396
Epoch 40/100
181/181 [==============================] - 87s 482ms/step - loss: 0.4500 - accuracy: 0.8792 - val_loss: 0.3652 - val_accuracy: 0.9010

Epoch 00040: val_accuracy did not improve from 0.91396
Epoch 41/100
181/181 [==============================] - 85s 471ms/step - loss: 0.4858 - accuracy: 0.8786 - val_loss: 0.3818 - val_accuracy: 0.9026

Epoch 00041: val_accuracy did not improve from 0.91396
Epoch 42/100
181/181 [==============================] - 85s 468ms/step - loss: 0.4446 - accuracy: 0.8755 - val_loss: 0.3742 - val_accuracy: 0.9091

Epoch 00042: val_accuracy did not improve from 0.91396
Epoch 43/100
181/181 [==============================] - 85s 471ms/step - loss: 0.3867 - accuracy: 0.8897 - val_loss: 0.3534 - val_accuracy: 0.9091

Epoch 00043: val_accuracy did not improve from 0.9139

In [35]:
df_train.columns

Index([                               'images',
       ('atypical melanocytic proliferation',),
                      ('cafe-au-lait macule',),
                              ('lentigo NOS',),
                      ('lichenoid keratosis',),
                                 ('melanoma',),
                                    ('nevus',),
                     ('seborrheic keratosis',),
                            ('solar lentigo',),
                                  ('unknown',)],
      dtype='object')

In [36]:
test_datagen = ImageDataGenerator(rescale=1./255)

data = test_datagen.flow_from_dataframe(dataframe = df_train.iloc[:100,:],
      x_col="images", class_mode=None, 
      batch_size=1, target_size=(224,224), shuffle=False)

predictions= model.predict_generator(data, max_queue_size=64, workers = 32, verbose=1)

Found 100 validated image filenames.


/home/deeplab/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


100/100 [==============================] - 4s 19ms/step


In [50]:
predictions[12, :]

array([7.1246497e-05, 2.7488306e-05, 1.1581112e-04, 1.0872406e-04,
       2.2896517e-03, 9.0283650e-01, 1.0275161e-04, 5.7522509e-05,
       9.4390258e-02], dtype=float32)

In [49]:
df_train.iloc[12, :]

images                                   train/ISIC_0076995.jpg
(atypical melanocytic proliferation,)                       0.0
(cafe-au-lait macule,)                                      0.0
(lentigo NOS,)                                              0.0
(lichenoid keratosis,)                                      0.0
(melanoma,)                                                 0.0
(nevus,)                                                    1.0
(seborrheic keratosis,)                                     0.0
(solar lentigo,)                                            0.0
(unknown,)                                                  0.0
Name: 12, dtype: object